- Remember to have 6 colors selected already
- Open the website floattheturn.com

In [1]:
import os
import sys
import ast
import math
import itertools
import functools
import subprocess
import numpy as np
import pandas as pd
from copy import deepcopy
from collections import defaultdict


# RFI
lj_rfi = "55+,AJo+,KQo,A9s+,K9s+,Q9s+,J9s+,T9s"
hj_rfi = "22+,AJo+,KJo+,QJo,A9s+,K9s+,Q9s+,J9s+,T9s"
co_rfi = "22+,ATo+,KTo+,QTo+,JTo,A5s+,K9s+,Q9s+,J9s+,T9s"
bn_rfi = "22+,A9o+,K9o+,Q9o+,J9o+,T9o,A2s+,K8s+,Q8s+,J8s+,T8s+,97s+,86s+,75s+,64s+,53s+,43s"
sb_rfi = "22+,A5o+,K9o+,Q9o+,J9o+,T9o,A2s+,K8s+,Q8s+,J8s+,T8s+,97s+,86s+,75s+,64s+,53s+,43s"

# vs RFI
vsRFI_bb_vs_lj__raise = "QQ+,AKo,AQs+"
vsRFI_bb_vs_hj__raise = "QQ+,AKo,AJs+,KQs"
vsRFI_bb_vs_co__raise = "JJ+,AKo,AJs+,KQs,QJs,JTs"
vsRFI_bb_vs_bn__raise = "TT+,AQo+,KQo,ATs+,KTs+,QTs+,JTs"
vsRFI_bb_vs_sb__raise = "99+,AQo+,KQo,A9s+,K9s+,Q9s+,J9s+,T9s"

vsRFI_bb_vs_lj__call = "JJ-22,AQo-AJo,KQo-KJo,QJo,AJs-A9s,KQs-K9s,QJs-Q9s,JTs-J9s,T9s"
vsRFI_bb_vs_hj__call = "JJ-22,AQo-ATo,KQo-KTo,QJo-QTo,JTo,ATs-A5s,KJs-K9s,QJs-Q9s,JTs-J9s,T9s"
vsRFI_bb_vs_co__call = "TT-22,AQo-A9o,KQo-K9o,QJo-Q9o,JTo-J9o,T9o,ATs-A2s,KJs-K8s,QTs-Q8s,J9s-J8s,T9s-T8s,98s-97s,87s-86s,76s-75s,65s-64s,54s-53s,43s"
vsRFI_bb_vs_bn__call = "99-22,AJo-A5o,KJo-K9o,QJo-Q9o,JTo-J9o,T9o,A9s-A2s,K9s-K8s,Q9s-Q8s,J9s-J8s,T9s-T8s,98s-97s,87s-86s,76s-75s,65s-64s,54s-53s,43s"
vsRFI_bb_vs_sb__call = "88-22,AJo-A8o,KJo-K8o,QJo-Q8o,JTo-J8o,T9o-T8o,98o,A8s-A2s,K8s-K4s,Q8s-Q4s,J8s-J4s,T8s-T4s,98s-94s,87s-84s,76s-74s,65s-64s,54s-53s,43s-42s,32s"

# vs RFI
vsRFI_sb_vs_bn__raise = lj_rfi
vsRFI_other__raise = "AA-QQ,AKo,AKs,A5s-A4s"
vsRFI_other__call = "JJ-77,AQo,AQs-ATs,KQs-KTs,QJs-QTs,JTs"

# RFIvs3B
RFIvs3B_other__raise = "AA-QQ,AKo,AKs,A5s-A4s"
RFIvs3B_other__call = "JJ-77,AQo,AKs-ATs,KQs-KTs,QJs-QTs,JTs"

# Cold 4B
cold4B = "AA-KK"

# Input: range
m = {
    "A": 14, "K": 13, "Q": 12, "J": 11, "T": 10, "9": 9,
    "8": 8, "7": 7, "6": 6, "5": 5, "4": 4, "3": 3, "2": 2
}
m2 = {14: 'A', 13: 'K', 12: 'Q', 11: 'J', 10: 'T',
      9: '9', 8: '8', 7: '7', 6: '6', 5: '5',
      4: '4', 3: '3', 2: '2'}
def range_to_hands(c_range="JJ+,AJs+,KQs,AKo"):
    temp = c_range.split(",")

    pps = []
    pp = temp[0]
    if "+" in pp:
        for i in range(14,m[pp[0]]-1,-1):
            pps.append([i, i])
    elif "-" in pp:
        for i in range(m[pp[0]],m[pp[-1]]-1,-1):
            pps.append([i, i])
    else:
        pps.append([m[pp[0]], m[pp[0]]])

    ss = []
    temp_s = [x for x in temp if "s" in x]
    for s in temp_s:
        if "+" in s:
            for i in range(m[s[0]]-1,m[s[1]]-1,-1):
                ss.append([m[s[0]], i])
        elif "-" in s:
            for i in range(m[s[1]],m[s[5]]-1,-1):
                ss.append([m[s[0]], i])
        else:
            ss.append([m[s[0]], m[s[1]]])

    os = []
    temp_o = [x for x in temp if "o" in x]
    for o in temp_o:
        if "+" in o:
            for i in range(m[o[0]]-1,m[o[1]]-1,-1):
                os.append([m[o[0]], i])
        elif "-" in o:
            for i in range(m[o[1]],m[o[5]]-1,-1):
                os.append([m[o[0]], i])
        else:
            os.append([m[o[0]], m[o[1]]])
    # Output: [[2,2]], [[14,13]], [[14,13]]
        # PP, Suited, Offsuit
    return pps, ss, os


cat1_rankings = ["set", "trips", "two pair", "overpair 9+", "any overpair", "TP J-kicker",
                 "TP K-kicker", "TP any kicker"]
cat2_nonpaired_rankings = ["top pair bad kicker", "middle pair", "bottom pair", "PP below middle pair",
                           "AJ high", "KQ high", "KJ high bdfd", "K8 high bdfd", ]
cat2_paired_rankings = ["Ace high", "PP below top card", "KQ high", "all"]
cat3_rankings = ["FD", "OESD", "Gutshot", "3 to a straight not all from low end",
                 "3 to a straight low end bdfd", "3 to a straight low end",
                 "5 cards within 7 values with bdfd", "Q- high bdfd",
                 "3 cards within 4 values as overcards", "A- bdfd"]
first_cat4_pp_rankings = ["JJ", "TT", "99", "88", "77", "66", "55", "44", "33", "22"]



def my_hands_cat1_level_x_and_above(x):
    result = [[], [], []]
    if x >= 1:
        result[1] += my_hands_s_straight
        result[2] += my_hands_o_straight
    if x >= 2:
        result[0] += my_hands_pp_sets
    if x >= 3:
        result[1] += my_hands_s_trips
        result[2] += my_hands_o_trips
    if x >= 4:
        result[1] += my_hands_s_two_pair
        result[2] += my_hands_o_two_pair
    if x >= 5:
        result[0] += my_hands_pp_overpair_9plus
    if x >= 6:
        result[0] += my_hands_pp_any_overpair
    if x >= 7:
        result[1] += my_hands_s_tp_k_kicker
        result[2] += my_hands_o_tp_k_kicker
    if x >= 8:
        result[1] += my_hands_s_tp_j_kicker
        result[2] += my_hands_o_tp_j_kicker
    if x >= 9:
        result[1] += my_hands_s_tp_any_kicker
        result[2] += my_hands_o_tp_any_kicker

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Return result
    my_hands_cat1 = result
    return my_hands_cat1

# Performance improvement by filtering out cat1 from hands already, but would also need a copy of hands
def my_hands_cat2_level_x_and_above(x, my_hands_cat1):
    result = [[], [], []]
    if x >= 1:
        # Cat 1
        result[1] += my_hands_s_straight
        result[2] += my_hands_o_straight
        result[0] += my_hands_pp_sets
        result[1] += my_hands_s_trips
        result[2] += my_hands_o_trips
        result[1] += my_hands_s_two_pair
        result[2] += my_hands_o_two_pair
        result[0] += my_hands_pp_overpair_9plus
        result[0] += my_hands_pp_any_overpair
        result[1] += my_hands_s_tp_k_kicker
        result[2] += my_hands_o_tp_k_kicker
        result[1] += my_hands_s_tp_j_kicker
        result[2] += my_hands_o_tp_j_kicker
        result[1] += my_hands_s_tp_any_kicker
        result[2] += my_hands_o_tp_any_kicker

        # Cat 2
        result[1] += my_hands_s_tp_bad_kicker
        result[2] += my_hands_o_tp_bad_kicker
    if x >= 2:
        result[1] += my_hands_s_middle_pair
        result[2] += my_hands_o_middle_pair
    if x >= 3:
        result[0] += my_hands_pp_below_top_pair
    if x >= 4:
        result[1] += my_hands_s_bottom_pair
        result[2] += my_hands_o_bottom_pair
    if x >= 5:
        result[1] += my_hands_s_aj_high
        result[2] += my_hands_o_aj_high
    if x >= 6:
        result[0] += my_hands_pp_below_middle_pair
    if x >= 7:
        result[1] += my_hands_s_kq_high
        result[2] += my_hands_o_kq_high
    if x >= 8:
        result[0] += my_hands_pp_below_bottom_pair
    if x >= 9:
        result[1] += my_hands_s_kj_high
        result[2] += my_hands_o_kj_high
    if x >= 10:
        result[1] += my_hands_s_k8_high
        result[2] += my_hands_o_k8_high

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Interim
    cat1_unique_pp = [x for (x,y) in my_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in my_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in my_hands_cat1[2]]

    # Remove cat1 from these cat2s
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o]

    # Return result
    my_hands_cat2 = result
    return my_hands_cat2

# Performance improvement by filtering out cat1+cat2 from hands already, but would also need a copy of hands
def my_hands_cat3_level_x_and_above(x, my_hands_cat1, my_hands_cat2):
    bdfd_result = [[], [], []]
    other_result = [[], [], []]
    result = [[], [], []]
    if x >= 1:
        other_result[0] += my_hands_pp_fd
        other_result[1] += my_hands_s_fd
        other_result[2] += my_hands_o_fd
    if x >= 2:
        other_result[0] += my_hands_pp_oesd
        other_result[1] += my_hands_s_oesd
        other_result[2] += my_hands_o_oesd
    if x >= 3:
        other_result[0] += my_hands_pp_gutshot
        other_result[1] += my_hands_s_gutshot
        other_result[2] += my_hands_o_gutshot
    if x >= 4:
        other_result[1] += my_hands_s_3_to_straight_not_all_from_low_end
        other_result[2] += my_hands_o_3_to_straight_not_all_from_low_end
    if x >= 5:
        bdfd_result[1] += my_hands_s_3_to_straight_low_end_bdfd
        bdfd_result[2] += my_hands_o_3_to_straight_low_end_bdfd
    if x >= 6:
        other_result[1] += my_hands_s_3_to_straight_low_end
        other_result[2] += my_hands_o_3_to_straight_low_end
    if x >= 7:
        bdfd_result[1] += my_hands_s_5_unique_cards_within_7_values_bdfd
        bdfd_result[2] += my_hands_o_5_unique_cards_within_7_values_bdfd
    if x >= 8:
        bdfd_result[0] += my_hands_pp_q_minus_bdfd
        bdfd_result[1] += my_hands_s_q_minus_bdfd
        bdfd_result[2] += my_hands_o_q_minus_bdfd
    if x >= 9:
        other_result[1] += my_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
        other_result[2] += my_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
    if x >= 10:
        bdfd_result[0] += my_hands_pp_a_minus_bdfd
        bdfd_result[1] += my_hands_s_a_minus_bdfd
        bdfd_result[2] += my_hands_o_a_minus_bdfd

    # Remove duplicates within bdfd hands
    bdfd_result[0].sort(reverse=True)
    bdfd_result[1].sort(reverse=True)
    bdfd_result[2].sort(reverse=True)
    bdfd_result[0] = list(k for k,_ in itertools.groupby(bdfd_result[0]))
    bdfd_result[1] = list(k for k,_ in itertools.groupby(bdfd_result[1]))
    bdfd_result[2] = list(k for k,_ in itertools.groupby(bdfd_result[2]))

    # Add all together
    result[0] = bdfd_result[0] + other_result[0]
    result[1] = bdfd_result[1] + other_result[1]
    result[2] = bdfd_result[2] + other_result[2]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] += val[1]
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] += val[1]
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] += val[1]
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Interim
    cat1_unique_pp = [x for (x,y) in my_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in my_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in my_hands_cat1[2]]
    cat2_unique_pp = [x for (x,y) in my_hands_cat2[0]]
    cat2_unique_s = [x for (x,y) in my_hands_cat2[1]]
    cat2_unique_o = [x for (x,y) in my_hands_cat2[2]]

    # Remove cat1 and cat2
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp and x not in cat2_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s and x not in cat2_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o and x not in cat2_unique_o]

    # Add cat2 hands
    if x >= 11:
        result[1] += [(x,y) for (x,y) in my_hands_s_k8_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_k8_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 12:
        result[1] += [(x,y) for (x,y) in my_hands_s_kj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_kj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 13:
        result[0] += [(x,y) for (x,y) in my_hands_pp_below_bottom_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 14:
        result[1] += [(x,y) for (x,y) in my_hands_s_kq_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_kq_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 15:
        result[0] += [(x,y) for (x,y) in my_hands_pp_below_middle_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    # Add cat4 hands
    if x >= 16:
        remaining_cat2_type_hands_pp = [x for (x,y) in my_hands_pp_below_top_pair]
        remaining_cat2_type_hands_s = [x for (x,y) in my_hands_s_aj_high] + [x for (x,y) in my_hands_s_bottom_pair] + [x for (x,y) in my_hands_s_middle_pair] + [x for (x,y) in my_hands_s_tp_bad_kicker]
        remaining_cat2_type_hands_o = [x for (x,y) in my_hands_o_aj_high] + [x for (x,y) in my_hands_o_bottom_pair] + [x for (x,y) in my_hands_o_middle_pair] + [x for (x,y) in my_hands_o_tp_bad_kicker]
        result[0] += [(x, 6) for x in my_hands[0] if x not in cat1_unique_pp and x not in cat2_unique_pp and x not in remaining_cat2_type_hands_pp]
        result[1] += [(x, 4) for x in my_hands[1] if x not in cat1_unique_s and x not in cat2_unique_s and x not in remaining_cat2_type_hands_s]
        result[2] += [(x, 12) for x in my_hands[2] if x not in cat1_unique_o and x not in cat2_unique_o and x not in remaining_cat2_type_hands_o]
    # Add cat2 hands with pairs
    if x >= 17:
        result[1] += [(x,y) for (x,y) in my_hands_s_aj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_aj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 18:
        result[1] += [(x,y) for (x,y) in my_hands_s_bottom_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_bottom_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 19:
        result[0] += [(x,y) for (x,y) in my_hands_pp_below_top_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 20:
        result[1] += [(x,y) for (x,y) in my_hands_s_middle_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_middle_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 21:
        result[1] += [(x,y) for (x,y) in my_hands_s_tp_bad_kicker if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_tp_bad_kicker if x not in cat1_unique_o and x not in cat2_unique_o]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Return results
    my_hands_cat3 = result
    return my_hands_cat3


def opponents_hands_cat1_level_x_and_above(x):
    result = [[], [], []]
    if x >= 1:
        result[1] += opponents_hands_s_straight
        result[2] += opponents_hands_o_straight
    if x >= 2:
        result[0] += opponents_hands_pp_sets
    if x >= 3:
        result[1] += opponents_hands_s_trips
        result[2] += opponents_hands_o_trips
    if x >= 4:
        result[1] += opponents_hands_s_two_pair
        result[2] += opponents_hands_o_two_pair
    if x >= 5:
        result[0] += opponents_hands_pp_overpair_9plus
    if x >= 6:
        result[0] += opponents_hands_pp_any_overpair
    if x >= 7:
        result[1] += opponents_hands_s_tp_k_kicker
        result[2] += opponents_hands_o_tp_k_kicker
    if x >= 8:
        result[1] += opponents_hands_s_tp_j_kicker
        result[2] += opponents_hands_o_tp_j_kicker
    if x >= 9:
        result[1] += opponents_hands_s_tp_any_kicker
        result[2] += opponents_hands_o_tp_any_kicker

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Return result
    opponents_hands_cat1 = resultsco
    return opponents_hands_cat1

# Performance improvement by filtering out cat1 from hands already, but would also need a copy of hands
def opponents_hands_cat2_level_x_and_above(x, opponents_hands_cat1):
    result = [[], [], []]
    if x >= 1:
        # Cat 1
        result[1] += opponents_hands_s_straight
        result[2] += opponents_hands_o_straight
        result[0] += opponents_hands_pp_sets
        result[1] += opponents_hands_s_trips
        result[2] += opponents_hands_o_trips
        result[1] += opponents_hands_s_two_pair
        result[2] += opponents_hands_o_two_pair
        result[0] += opponents_hands_pp_overpair_9plus
        result[0] += opponents_hands_pp_any_overpair
        result[1] += opponents_hands_s_tp_k_kicker
        result[2] += opponents_hands_o_tp_k_kicker
        result[1] += opponents_hands_s_tp_j_kicker
        result[2] += opponents_hands_o_tp_j_kicker
        result[1] += opponents_hands_s_tp_any_kicker
        result[2] += opponents_hands_o_tp_any_kicker

        # Cat 2
        result[1] += opponents_hands_s_tp_bad_kicker
        result[2] += opponents_hands_o_tp_bad_kicker
    if x >= 2:
        result[1] += opponents_hands_s_middle_pair
        result[2] += opponents_hands_o_middle_pair
    if x >= 3:
        result[0] += opponents_hands_pp_below_top_pair
    if x >= 4:
        result[1] += opponents_hands_s_bottom_pair
        result[2] += opponents_hands_o_bottom_pair
    if x >= 5:
        result[1] += opponents_hands_s_aj_high
        result[2] += opponents_hands_o_aj_high
    if x >= 6:
        result[0] += opponents_hands_pp_below_middle_pair
    if x >= 7:
        result[1] += opponents_hands_s_kq_high
        result[2] += opponents_hands_o_kq_high
    if x >= 8:
        result[0] += opponents_hands_pp_below_bottom_pair
    if x >= 9:
        result[1] += opponents_hands_s_kj_high
        result[2] += opponents_hands_o_kj_high
    if x >= 10:
        result[1] += opponents_hands_s_k8_high
        result[2] += opponents_hands_o_k8_high

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Interim
    cat1_unique_pp = [x for (x,y) in opponents_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in opponents_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in opponents_hands_cat1[2]]

    # Remove cat1 from these cat2s
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o]

    # Return result
    opponents_hands_cat2 = result
    return opponents_hands_cat2

# Performance improvement by filtering out cat1+cat2 from hands already, but would also need a copy of hands
def opponents_hands_cat3_level_x_and_above(x, opponents_hands_cat1, opponents_hands_cat2, skip_4_to_10_and_13_to_15=True):
    bdfd_result = [[], [], []]
    other_result = [[], [], []]
    result = [[], [], []]
    if x >= 1:
        other_result[0] += opponents_hands_pp_fd
        other_result[1] += opponents_hands_s_fd
        other_result[2] += opponents_hands_o_fd
    if x >= 2:
        other_result[0] += opponents_hands_pp_oesd
        other_result[1] += opponents_hands_s_oesd
        other_result[2] += opponents_hands_o_oesd
    if x >= 3:
        other_result[0] += opponents_hands_pp_gutshot
        other_result[1] += opponents_hands_s_gutshot
        other_result[2] += opponents_hands_o_gutshot
    if x >= 4 and not skip_4_to_10_and_13_to_15:
        other_result[1] += opponents_hands_s_3_to_straight_not_all_from_low_end
        other_result[2] += opponents_hands_o_3_to_straight_not_all_from_low_end
    if x >= 5 and not skip_4_to_10_and_13_to_15:
        bdfd_result[1] += opponents_hands_s_3_to_straight_low_end_bdfd
        bdfd_result[2] += opponents_hands_o_3_to_straight_low_end_bdfd
    if x >= 6 and not skip_4_to_10_and_13_to_15:
        other_result[1] += opponents_hands_s_3_to_straight_low_end
        other_result[2] += opponents_hands_o_3_to_straight_low_end
    if x >= 7 and not skip_4_to_10_and_13_to_15:
        bdfd_result[1] += opponents_hands_s_5_unique_cards_within_7_values_bdfd
        bdfd_result[2] += opponents_hands_o_5_unique_cards_within_7_values_bdfd
    if x >= 8 and not skip_4_to_10_and_13_to_15:
        bdfd_result[0] += opponents_hands_pp_q_minus_bdfd
        bdfd_result[1] += opponents_hands_s_q_minus_bdfd
        bdfd_result[2] += opponents_hands_o_q_minus_bdfd
    if x >= 9:
        other_result[1] += opponents_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
        other_result[2] += opponents_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
    if x >= 10 and not skip_4_to_10_and_13_to_15:
        bdfd_result[0] += opponents_hands_pp_a_minus_bdfd
        bdfd_result[1] += opponents_hands_s_a_minus_bdfd
        bdfd_result[2] += opponents_hands_o_a_minus_bdfd

    # Remove duplicates within bdfd hands
    bdfd_result[0].sort(reverse=True)
    bdfd_result[1].sort(reverse=True)
    bdfd_result[2].sort(reverse=True)
    bdfd_result[0] = list(k for k,_ in itertools.groupby(bdfd_result[0]))
    bdfd_result[1] = list(k for k,_ in itertools.groupby(bdfd_result[1]))
    bdfd_result[2] = list(k for k,_ in itertools.groupby(bdfd_result[2]))

    # Add all together
    result[0] = bdfd_result[0] + other_result[0]
    result[1] = bdfd_result[1] + other_result[1]
    result[2] = bdfd_result[2] + other_result[2]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] += val[1]
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] += val[1]
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] += val[1]
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Interim
    cat1_unique_pp = [x for (x,y) in opponents_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in opponents_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in opponents_hands_cat1[2]]
    cat2_unique_pp = [x for (x,y) in opponents_hands_cat2[0]]
    cat2_unique_s = [x for (x,y) in opponents_hands_cat2[1]]
    cat2_unique_o = [x for (x,y) in opponents_hands_cat2[2]]

    # Remove cat1 and cat2
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp and x not in cat2_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s and x not in cat2_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o and x not in cat2_unique_o]

    # Add cat2 hands
    if x >= 11 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_k8_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_k8_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 12 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_kj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_kj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 13 and not skip_4_to_10_and_13_to_15:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_bottom_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 14 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_kq_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_kq_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 15 and not skip_4_to_10_and_13_to_15:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_middle_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    # Add cat4 hands
    if x >= 16:
        remaining_cat2_type_hands_pp = [x for (x,y) in opponents_hands_pp_below_bottom_pair] + [x for (x,y) in opponents_hands_pp_below_middle_pair] + [x for (x,y) in opponents_hands_pp_below_top_pair]
        remaining_cat2_type_hands_s = [x for (x,y) in opponents_hands_s_k8_high] + [x for (x,y) in opponents_hands_s_kj_high] + [x for (x,y) in opponents_hands_s_kq_high] + [x for (x,y) in opponents_hands_s_aj_high] + [x for (x,y) in opponents_hands_s_bottom_pair] + [x for (x,y) in opponents_hands_s_middle_pair] + [x for (x,y) in opponents_hands_s_tp_bad_kicker]
        remaining_cat2_type_hands_o = [x for (x,y) in opponents_hands_o_k8_high] + [x for (x,y) in opponents_hands_o_kj_high] + [x for (x,y) in opponents_hands_o_kq_high] + [x for (x,y) in opponents_hands_o_aj_high] + [x for (x,y) in opponents_hands_o_bottom_pair] + [x for (x,y) in opponents_hands_o_middle_pair] + [x for (x,y) in opponents_hands_o_tp_bad_kicker]
        result[0] += [(x, 6) for x in opponents_hands[0] if x not in cat1_unique_pp and x not in cat2_unique_pp and x not in remaining_cat2_type_hands_pp]
        result[1] += [(x, 4) for x in opponents_hands[1] if x not in cat1_unique_s and x not in cat2_unique_s and x not in remaining_cat2_type_hands_s]
        result[2] += [(x, 12) for x in opponents_hands[2] if x not in cat1_unique_o and x not in cat2_unique_o and x not in remaining_cat2_type_hands_o]
    # Add cat2 hands with pairs
    if x >= 17:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_aj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_aj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 18:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_bottom_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_bottom_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 19:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_top_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 20:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_middle_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_middle_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 21:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_tp_bad_kicker if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_tp_bad_kicker if x not in cat1_unique_o and x not in cat2_unique_o]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Return results
    opponents_hands_cat3 = result
    return opponents_hands_cat3





opponent_unraised_strategy = None # To be defined later; changes by flop
opponent_raised_strategy = {
    'cat1': {1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6},
    'cat2': {1: 3, 2: 5, 3: 6, 4: 6, 5: 7, 6: 7, 7: 7},
    'cat3': {1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 5, 7: 5},
}
opponent_reraised_strategy = {
    'cat1': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
    'cat2': {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3},
    'cat3': {1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3},
}
opponent_strategy = None
def get_flop_type_number():
    return \
    7 if flop[0] == flop[1] or flop[1] == flop[2] else \
    1 if flop[0] >= 13 and flop[1] >= 13 else \
    2 if flop[0] >= 13 and flop[1] >= 9 else \
    3 if flop[0] >= 13 else \
    4 if flop[0] >= 10 and flop[1] >= 9 else \
    5 if flop[0] >= 10 else \
    6
def get_opponent_situation(bets):
    return \
    "oop_open" if bets == 0 and my_position_ip == True else \
    "oop_vs_cb" if bets == 1 and my_position_ip == True else \
    "oop_vs_br" if bets >= 2 and my_position_ip == True else \
    "ip_vs_c" if bets == 0 and my_position_ip == False else \
    "ip_vs_b" if bets == 1 and my_position_ip == False else \
    "ip_vs_cbr"

def combine_hands(hands1, hands2):
    hands = [[], [], []]
    for i in range(3):
        hands[i] = hands1[i] + hands2[i]
    return hands

def get_cat4_hands(all_hands_before_action_w_combos, cat1_hands_for_action, cat2_hands_for_action, cat3_hands_for_action):
    hands = [[], [], []]
    temp_cat3_hands = deepcopy(cat3_hands_for_action)

    # Flip sign (for subtraction)
    for i in range(3):
        temp_cat3_hands[i] = [(x, -1*y) for (x, y) in temp_cat3_hands[i]]

    # Combine (for subtraction)
    result = combine_hands(all_hands_before_action_w_combos, temp_cat3_hands)

    # Subtraction
    for i in range(3):
        groupby_dict = defaultdict(int)
        for val in result[i]:
            groupby_dict[tuple(val[0])] += val[1]
        result[i] = [(sorted(list(x), reverse=True), max(0, min(y, 6 if i == 0 else 4 if i == 1 else 12))) for (x,y) in groupby_dict.items()]
        result[i] = [(x,y) for (x,y) in result[i] if y != 0 and x not in [x for (x,y) in cat1_hands_for_action[i]] and x not in [x for (x,y) in cat2_hands_for_action[i]]]
    return result


#####

all_flops = []
for rank1 in range(14,1,-1):
    for rank2 in range(14,1,-1):
        for rank3 in range(14,1,-1):
            if rank1 >= rank2 and rank2 >= rank3:
                all_flops.append([rank1, rank2, rank3])

# # Start from a spot
# flops = flops[406:]


range_names = ["lj_rfi",
"hj_rfi",
"co_rfi",
"bn_rfi",
"sb_rfi",
"vsRFI_bb_vs_lj__raise",
"vsRFI_bb_vs_hj__raise",
"vsRFI_bb_vs_co__raise",
"vsRFI_bb_vs_bn__raise",
"vsRFI_bb_vs_sb__raise",
"vsRFI_bb_vs_lj__call",
"vsRFI_bb_vs_hj__call",
"vsRFI_bb_vs_co__call",
"vsRFI_bb_vs_bn__call",
"vsRFI_bb_vs_sb__call",
"vsRFI_sb_vs_bn__raise",
"vsRFI_other__raise",
"vsRFI_other__call",
"RFIvs3B_other__raise",
"RFIvs3B_other__call",
"cold4B"]
my_ranges = [lj_rfi,
hj_rfi,
co_rfi,
bn_rfi,
sb_rfi,
vsRFI_bb_vs_lj__raise,
vsRFI_bb_vs_hj__raise,
vsRFI_bb_vs_co__raise,
vsRFI_bb_vs_bn__raise,
vsRFI_bb_vs_sb__raise,
vsRFI_bb_vs_lj__call,
vsRFI_bb_vs_hj__call,
vsRFI_bb_vs_co__call,
vsRFI_bb_vs_bn__call,
vsRFI_bb_vs_sb__call,
vsRFI_sb_vs_bn__raise,
vsRFI_other__raise,
vsRFI_other__call,
RFIvs3B_other__raise,
RFIvs3B_other__call,
cold4B]
range_names_print = ['LJ RFI', 'HJ RFI', 
                    'CO RFI', 'BN RFI', 
                    'SB RFI', 
                    'Villain LJ RFI and You BB 3Bet',
                    'Villain HJ RFI and You BB 3Bet',
                    'Villain CO RFI and You BB 3Bet',
                    'Villain BN RFI and You BB 3Bet',
                    'Villain SB RFI and You BB 3Bet',
                    'Villain LJ RFI and You BB Call',
                    'Villain HJ RFI and You BB Call',
                    'CO RFI and You BB Call',
                    'Villain BN RFI and You BB Call',
                    'Villain SB RFI and You BB Call',
                    'Villain BN RFI and You SB 3Bet',
                    'Villain RFI and You 3Bet (Not BB)',
                    'Villain RFI and You Call (Not BB)',
                    'You RFI, get 3Bet and you 4Bet',
                    'You RFI, get 3Bet and you Call',
                    'You Cold 4Bet',
]

# Estimation:
my_position_ips = [False, False, 
                    True, True, 
                    False, 
                    False,
                   False,
                   False,
                   False,
                   False,
                   False,
                   False,
                   False,
                   False,
                   False,
                   False,
                   True,
                   True,
                   True,
                   True,
                   False
                  ]

my_pfrs = [True, True, 
                    True, True, 
                    True, 
                    True,
           True,
           True,
           True,
           True,
           False,
           False,
           False,
           False,
           False,
           True,
           True,
           False,
           True,
           False,
           True,
]



# Might add this later:
# opponents_ranges = [vsRFI_bb_vs_lj__raise, vsRFI_bb_vs_lj__call, vsRFI_hj_vs_lj__raise, vsRFI_co_vs_lj__raise, vsRFI_bn_vs_lj__raise, vsRFI_sb_vs_lj__raise, vsRFI_hj_vs_lj__call, vsRFI_co_vs_lj__call, vsRFI_bn_vs_lj__call, vsRFI_sb_vs_lj__call, vsRFI_bb_vs_hj__raise, vsRFI_bb_vs_hj__call, vsRFI_co_vs_hj__raise, vsRFI_bn_vs_hj__raise, vsRFI_sb_vs_hj__raise, vsRFI_co_vs_hj__call, vsRFI_bn_vs_hj__call, vsRFI_sb_vs_hj__call, vsRFI_bb_vs_co__raise, vsRFI_bb_vs_co__call, vsRFI_bn_vs_co__raise, vsRFI_sb_vs_co__raise, vsRFI_bn_vs_co__call, vsRFI_sb_vs_co__call, vsRFI_bb_vs_bn__raise, vsRFI_bb_vs_bn__call, vsRFI_sb_vs_bn__raise, vsRFI_bb_vs_sb__raise, vsRFI_bb_vs_sb__call, RFIvs3B_lj_vs_blinds_call, lj_rfi, RFIvs3B_lj_vs_hjco_call, RFIvs3B_lj_vs_hjco_call, RFIvs3B_lj_vs_bn_call, RFIvs3B_lj_vs_blinds_call, lj_rfi, lj_rfi, lj_rfi, lj_rfi, RFIvs3B_hj_vs_ahead_call, hj_rfi, RFIvs3B_hj_vs_ahead_call, RFIvs3B_hj_vs_ahead_call, RFIvs3B_hj_vs_ahead_call, hj_rfi, hj_rfi, hj_rfi, RFIvs3B_co_vs_blinds_call, co_rfi, RFIvs3B_co_vs_bn_call, RFIvs3B_co_vs_blinds_call, co_rfi, co_rfi, RFIvs3B_bnsb_vs_ahead_call, bn_rfi, RFIvs3B_bnsb_vs_ahead_call, RFIvs3B_bnsb_vs_ahead_call, sb_rfi]




In [2]:
my_hands_string = "AA,KK"
opponents_hands_string = "AA,55,44"
final_flop_string = "2c2h3d"

command = "source /Users/petermyers/Documents/pbots_calc-master/venv/bin/activate; /Users/petermyers/Documents/pbots_calc-master/python/calculator.sh {}:{} {}".format(my_hands_string, opponents_hands_string, final_flop_string)
process = subprocess.Popen(command,stdout=subprocess.PIPE, shell=True)
raw_equity = ast.literal_eval(process.communicate()[0].strip().decode("utf-8"))[0][1]
raw_equity


0.7228739002932552

In [18]:
import random
ranks = list(range(2,15))
flop = list([x for x in sorted([random.choice(ranks), random.choice(ranks), random.choice(ranks)], reverse=True)])
turn = random.choice(ranks)
river = random.choice(ranks)
turn_suit = random.choice(['suited', 'offsuit', 'offsuit', 'offsuit'])
river_suit = random.choice(['suited', 'offsuit', 'offsuit', 'offsuit'])
flop_action = random.choice(["You raise", "You bet", "You call", "Both check"])
turn_action = random.choice(["You raise", "You bet", "You call", "Both check"])
river_action = random.choice(["You raise", "You bet", "You call", "Both check"])
tones = random.choice([1] + [2]*13 + [3]*6)
indices_to_randomly_choose_from = [0,1,2,3,4,5]*5 + [6,7,8,9,10,11,12,13,14,15,16]*3 + list(range(len(range_names)))
hand_index = random.choice(indices_to_randomly_choose_from)

# Initialize variables using index
my_range = my_ranges[hand_index]
my_position_ip = my_position_ips[hand_index]
my_pfr = my_pfrs[hand_index]

# Helpers
board_type = "rainbow" if tones == 3 else "two-tone" if tones == 2 else "monotone" if tones == 1 else "Invalid"
is_paired = 1 if flop[0] == flop[1] or flop[1] == flop[2] else 0
paired_value = 0 if not is_paired else flop[0] if flop[0] == flop[1] else flop[1]
my_hands = range_to_hands(my_range)
if board_type == "monotone" and is_paired:
    print("Invalid")
if board_type == "two-tone" and flop[0] == flop[2]:
    print("Invalid")
    
# Print
print(range_names_print[hand_index])
print("Flop:", "".join([m2[x] for x in flop]))
print(board_type)
print("Flop Action:", flop_action)
print("Turn:", m2[turn], turn_suit)
print("Turn Action:", turn_action)
print("River:", m2[river], river_suit)
print("River Action:", river_action)

# My hands with combos
my_hands_with_combos = [[], [], []]
my_hands_with_combos[0] = [(x, 6) if x[0] not in flop else (x, 3) for x in my_hands[0]]
my_hands_with_combos[1] = [(x, 4) if x[0] not in flop and x[1] not in flop else (x, 2) if x[0] in flop and x[1] in flop else (x,3) for x in my_hands[1]]
my_hands_with_combos[2] = [(x, 12) if x[0] not in flop and x[1] not in flop else (x, 7) if x[0] in flop and x[1] in flop else (x,9) for x in my_hands[2]]





# Cat1
#### Assuming no flushes (monotone boards) for simplicity
my_hands_s_straight = [] if is_paired else [(x, 4) for x in my_hands[1] if len(set(x + flop)) == 5 and (max(x + flop) - min(x + flop) == 4 or max([1 if y == 14 else y for y in (x + flop)]) - min([1 if y == 14 else y for y in (x + flop)]) == 4)]
my_hands_o_straight = [] if is_paired else [(x, 12) for x in my_hands[2] if len(set(x + flop)) == 5 and (max(x + flop) - min(x + flop) == 4 or max([1 if y == 14 else y for y in (x + flop)]) - min([1 if y == 14 else y for y in (x + flop)]) == 4)]
my_hands_pp_sets = [(x, 3) for x in my_hands[0] if x[0] in flop]
my_hands_s_trips = [] if not is_paired else [(x, 2) for x in my_hands[1] if x[0] == paired_value or x[1] == paired_value]
my_hands_o_trips = [] if not is_paired else [(x, 6) for x in my_hands[2] if x[0] == paired_value or x[1] == paired_value]
# 2 combos most times, not 3; 7 more often than 6
my_hands_s_two_pair = [] if is_paired else [(x, 2) for x in my_hands[1] if x[0] in flop and x[1] in flop]
my_hands_o_two_pair = [] if is_paired else [(x, 7) for x in my_hands[2] if x[0] in flop and x[1] in flop]
my_hands_pp_overpair_9plus = [(x, 6) for x in my_hands[0] if x[0] > flop[0] and x[0] >= 9]
my_hands_pp_any_overpair = [(x, 6) for x in my_hands[0] if x[0] > flop[0]]
my_hands_s_tp_k_kicker = [(x, 3) for x in my_hands[1] if (x[0] == flop[0] and x[1] >= 13) or (x[1] == flop[0] and x[0] >= 13)]
my_hands_o_tp_k_kicker = [(x, 9) for x in my_hands[2] if (x[0] == flop[0] and x[1] >= 13) or (x[1] == flop[0] and x[0] >= 13)]
my_hands_s_tp_j_kicker = [(x, 3) for x in my_hands[1] if (x[0] == flop[0] and x[1] >= 11 and x[1] <= 12 and x[1] not in flop) or (x[1] == flop[0] and x[0] >= 11 and x[0] <= 12 and x[0] not in flop)]
my_hands_o_tp_j_kicker = [(x, 9) for x in my_hands[2] if (x[0] == flop[0] and x[1] >= 11 and x[1] <= 12 and x[1] not in flop) or (x[1] == flop[0] and x[0] >= 11 and x[0] <= 12 and x[0] not in flop)]
my_hands_s_tp_any_kicker = [(x, 3) for x in my_hands[1] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]
my_hands_o_tp_any_kicker = [(x, 9) for x in my_hands[2] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]

# Cat2 (flushdraws with high card hand might actually be part of cat3, but saying the combos are part of cat2)
my_hands_s_tp_bad_kicker = [(x, 3) for x in my_hands[1] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]
my_hands_o_tp_bad_kicker = [(x, 9) for x in my_hands[2] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]
my_hands_s_middle_pair = [(x, 3) for x in my_hands[1] if (x[0] == flop[1] and x[1] not in flop) or (x[1] == flop[1] and x[0] not in flop)]
my_hands_o_middle_pair = [(x, 9) for x in my_hands[2] if (x[0] == flop[1] and x[1] not in flop) or (x[1] == flop[1] and x[0] not in flop)]
my_hands_s_bottom_pair = [(x, 3) for x in my_hands[1] if (x[0] == flop[2] and x[1] not in flop) or (x[1] == flop[2] and x[0] not in flop)]
my_hands_o_bottom_pair = [(x, 9) for x in my_hands[2] if (x[0] == flop[2] and x[1] not in flop) or (x[1] == flop[2] and x[0] not in flop)]
my_hands_pp_below_top_pair = [(x, 6) for x in my_hands[0] if x[0] < flop[0] and x[0] > flop[1]]
my_hands_s_aj_high = [(x, 4) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and (x[0] == 14) and (x[1] > 10)]
my_hands_o_aj_high = [(x, 12) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and (x[0] == 14) and (x[1] > 10)]
my_hands_pp_below_middle_pair = [(x, 6) for x in my_hands[0] if x[0] < flop[1] and x[0] > flop[2]]
my_hands_s_kq_high = [(x, 4) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and ((x[0] == 13 and x[1] > 11) or (x[0] == 14))]
my_hands_o_kq_high = [(x, 12) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and ((x[0] == 13 and x[1] > 11) or (x[0] == 14))]
my_hands_pp_below_bottom_pair = [(x, 6) for x in my_hands[0] if x[0] < flop[2]]
my_hands_s_kj_high = [(x, 4) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] == 11)]
my_hands_o_kj_high = [(x, 12) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] == 11)]
my_hands_s_k8_high = [(x, 4) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] < 11 and x[1] >= 8)]
my_hands_o_k8_high = [(x, 12) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] < 11 and x[1] >= 8)]

# Cat3 paired
#### bdfd/fd/no_fd combos should not be counted twice; combine them smartly and carefully.
#### Assuming no suited flushdraw possible with pair on two-tone for simplicity
#### Might say a straight is a gutshot but that is fine because future logic
## Include the combos for that hand
if board_type == "two-tone":
    my_hands_pp_fd = []
    my_hands_s_fd = [(x, 1) for x in my_hands[1] if x[0] not in flop and x[1] not in flop]
    my_hands_o_fd = []
elif board_type == "rainbow":
    my_hands_pp_fd = []
    my_hands_s_fd = []
    my_hands_o_fd = []
else:
    my_hands_pp_fd = [(x, 3) for x in my_hands[0] if x[0] not in flop]
    my_hands_s_fd = []
    # If paired then ignore the flushdraw (anyway, just monotone; just makes things simpler)
    my_hands_o_fd = [(x, 6) for x in my_hands[2] if x[0] not in flop and x[1] not in flop]
#### Also added double gutshots.  Doing a bit of overcounting for oesd+pair, which is fine for estimation (should be 9).
my_hands_pp_oesd = [(x, 6) for x in my_hands[0] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 3) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 3) or (sorted(x+flop) == [3,4,5,7,14]) or (max(x+flop) - min(x+flop) == 6 and min(x+flop)+2 == sorted(x+flop)[1] and max(x+flop)-2 == sorted(x+flop)[-2])]
my_hands_s_oesd = [(x, 4) for x in my_hands[1] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 3) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 3) or (sorted(x+flop) == [3,4,5,7,14]) or (max(x+flop) - min(x+flop) == 6 and min(x+flop)+2 == sorted(x+flop)[1] and max(x+flop)-2 == sorted(x+flop)[-2])]
my_hands_o_oesd = [(x, 12) for x in my_hands[2] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 3) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 3) or (sorted(x+flop) == [3,4,5,7,14]) or (max(x+flop) - min(x+flop) == 6 and min(x+flop)+2 == sorted(x+flop)[1] and max(x+flop)-2 == sorted(x+flop)[-2])]
my_hands_pp_gutshot = [(x, 6) for x in my_hands[0] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 4) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 4) or (sorted(set([1 if y == 14 else y for y in (x + flop)] + [20,21,22]))[3] <= 5)]
my_hands_s_gutshot = [(x, 4) for x in my_hands[1] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 4) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 4) or (sorted(set([1 if y == 14 else y for y in (x + flop)] + [20,21,22]))[3] <= 5)]
my_hands_o_gutshot = [(x, 12) for x in my_hands[2] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 4) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 4) or (sorted(set([1 if y == 14 else y for y in (x + flop)] + [20,21,22]))[3] <= 5)]
#### Additional rule: 3 to a straight requires two cards from your hand not just one (that's how I want it to be)
my_hands_s_3_to_straight_not_all_from_low_end = [(x, 4) for x in my_hands[1] if (x[0] != 14) and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]+1 and x[1] == flop[0]-1) or (x[0] == flop[0]+2 and x[1] == flop[0]+1) or (x[0] == flop[1]+1 and x[1] == flop[1]-1) or (x[0] == flop[1]+2 and x[1] == flop[1]+1) or (x[0] == flop[2]+1 and x[1] == flop[2]-1) or (x[0] == flop[2]+2 and x[1] == flop[2]+1))]
my_hands_o_3_to_straight_not_all_from_low_end = [(x, 12) for x in my_hands[2] if (x[0] != 14) and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]+1 and x[1] == flop[0]-1) or (x[0] == flop[0]+2 and x[1] == flop[0]+1) or (x[0] == flop[1]+1 and x[1] == flop[1]-1) or (x[0] == flop[1]+2 and x[1] == flop[1]+1) or (x[0] == flop[2]+1 and x[1] == flop[2]-1) or (x[0] == flop[2]+2 and x[1] == flop[2]+1))]
if board_type == "two-tone":
    my_hands_s_3_to_straight_low_end_bdfd = [(x, 1) for x in my_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    my_hands_o_3_to_straight_low_end_bdfd = [(x, 6) for x in my_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
elif board_type == "rainbow":
    my_hands_s_3_to_straight_low_end_bdfd = [(x, 3) for x in my_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    my_hands_o_3_to_straight_low_end_bdfd = []
else:
    my_hands_s_3_to_straight_low_end_bdfd = []
    my_hands_o_3_to_straight_low_end_bdfd = []
if board_type == "two-tone":
    my_hands_s_3_to_straight_low_end = [(x, 4) for x in my_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    my_hands_o_3_to_straight_low_end = [(x, 12) for x in my_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
elif board_type == "rainbow":
    my_hands_s_3_to_straight_low_end = [(x, 4) for x in my_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    my_hands_o_3_to_straight_low_end = [(x, 12) for x in my_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
else:
    my_hands_s_3_to_straight_low_end = [(x, 4) for x in my_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    my_hands_o_3_to_straight_low_end = [(x, 12) for x in my_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
if board_type == "two-tone":
    my_hands_s_5_unique_cards_within_7_values_bdfd = [(x, 1) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and max(flop+x) - min(flop+x) <= 7]
    my_hands_o_5_unique_cards_within_7_values_bdfd = [(x, 6) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and max(flop+x) - min(flop+x) <= 7]
elif board_type == "rainbow":
    my_hands_s_5_unique_cards_within_7_values_bdfd = [(x, 3) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and max(flop+x) - min(flop+x) <= 7]
    my_hands_o_5_unique_cards_within_7_values_bdfd = []
else:
    my_hands_s_5_unique_cards_within_7_values_bdfd = []
    my_hands_o_5_unique_cards_within_7_values_bdfd = []
if board_type == "two-tone":
    my_hands_pp_q_minus_bdfd = [(x, 3) for x in my_hands[0] if (x[0] not in flop) and x[0] <= 12]
    my_hands_s_q_minus_bdfd = [(x, 1) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] <= 12]
    my_hands_o_q_minus_bdfd = [(x, 6) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and x[0] <= 12]
elif board_type == "rainbow":
    my_hands_pp_q_minus_bdfd = []
    my_hands_s_q_minus_bdfd = [(x, 3) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] <= 12]
    my_hands_o_q_minus_bdfd = []
else:
    my_hands_pp_q_minus_bdfd = []
    my_hands_s_q_minus_bdfd = []
    my_hands_o_q_minus_bdfd = []
#### 3 cards within 4 values with two overcards
my_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards = [(x, 4) for x in my_hands[1] if x[1] > flop[0] and ((max(flop + [x[1]]) - sorted(set(flop + [x[1]] + [-20,-19,-18]))[-3] <= 3) or (max(flop + x) - sorted(set(flop + x + [-20,-19,-18]))[-3] <= 3))]
my_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards = [(x, 12) for x in my_hands[2] if x[1] > flop[0] and ((max(flop + [x[1]]) - sorted(set(flop + [x[1]] + [-20,-19,-18]))[-3] <= 3) or (max(flop + x) - sorted(set(flop + x + [-20,-19,-18]))[-3] <= 3))]
if board_type == "two-tone":
    my_hands_pp_a_minus_bdfd = [(x, 3) for x in my_hands[0] if (x[0] not in flop) and x[0] > 12 and x[0] <= 14]
    my_hands_s_a_minus_bdfd = [(x, 1) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] > 12 and x[0] <= 14]
    my_hands_o_a_minus_bdfd = [(x, 6) for x in my_hands[2] if (x[0] not in flop and x[1] not in flop) and x[0] > 12 and x[0] <= 14]
elif board_type == "rainbow":
    my_hands_pp_a_minus_bdfd = []
    my_hands_s_a_minus_bdfd = [(x, 3) for x in my_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] > 12 and x[0] <= 14]
    my_hands_o_a_minus_bdfd = []
else:
    my_hands_pp_a_minus_bdfd = []
    my_hands_s_a_minus_bdfd = []
    my_hands_o_a_minus_bdfd = []





def count_hand_combos(hands):
    return sum([y for (x,y) in hands[0]])+sum([y for (x,y) in hands[1]])+sum([y for (x,y) in hands[2]])


best_score = 100000
best_param1 = None
best_param2 = None


# Set param1 as a constant
if my_position_ip and (my_pfr or is_paired or max(flop) <= 9):
    param1 = 7
else:
    param1 = 5

for param1 in [param1]:
    for param2 in [1,2,5,8,16,10]:
        # LOCK IN AT 8:
        my_hands_cat1 = my_hands_cat1_level_x_and_above(8)
        # RANGE FROM 5-10:
        my_hands_cat2 = my_hands_cat2_level_x_and_above(param1, my_hands_cat1)
        # CHOICES 1, 2, 5, 8, 10, 16:
        my_hands_cat3 = my_hands_cat3_level_x_and_above(param2, my_hands_cat1, my_hands_cat2)
        my_hands_cat4 = get_cat4_hands(my_hands_with_combos, my_hands_cat1, my_hands_cat2, my_hands_cat3)

        # USE CONSTRAINT OPTIMIZATION CODE OR BRUTE FORCE
        cat1_combos = count_hand_combos(my_hands_cat1)
        cat2_combos = count_hand_combos(my_hands_cat2)
        cat3_combos = count_hand_combos(my_hands_cat3)
        cat4_combos = count_hand_combos(my_hands_cat4)
        # score = (3000 if cat3_combos == 0 else abs(cat1_combos*2/cat3_combos)**10.3) + (0 if cat4_combos == 0 and cat2_combos == 0 else 1000 if cat2_combos == 0 else (cat4_combos/cat2_combos)*2.0)
        score = (3 if cat3_combos == 0 else abs((cat1_combos*2-cat3_combos)/cat3_combos)) # + (3 if cat4_combos == 0 else abs((cat4_combos-cat2_combos)/cat4_combos))*0.5
        
        if score <= best_score:
            best_score = score
            best_param1 = param1
            best_param2 = param2
        
        
my_hands_cat1 = my_hands_cat1_level_x_and_above(8)
# RANGE FROM 5-10:
my_hands_cat2 = my_hands_cat2_level_x_and_above(best_param1, my_hands_cat1)
# CHOICES 1, 2, 5, 8, 10, 16:
my_hands_cat3 = my_hands_cat3_level_x_and_above(best_param2, my_hands_cat1, my_hands_cat2)      
my_hands_cat4 = get_cat4_hands(my_hands_with_combos, my_hands_cat1, my_hands_cat2, my_hands_cat3)
cat1_combos = count_hand_combos(my_hands_cat1)
cat2_combos = count_hand_combos(my_hands_cat2)
cat3_combos = count_hand_combos(my_hands_cat3)
cat4_combos = count_hand_combos(my_hands_cat4)
print(cat1_combos, cat2_combos, cat3_combos, cat4_combos)
# print("cat3 combos needed:", cat1_combos*2 - cat3_combos)
# print("cat2 combos needed:", cat4_combos - cat2_combos)
print("Weakest bluff to use:", best_param2)



import re
import pyperclip as clip
import pyautogui as g
from time import sleep
import pyperclip as clip
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
g.FAILSAFE = True
sleep(2)
g.PAUSE = 0.0
g.size()
g.position()

def c():
    g.mouseDown()
    g.mouseUp()
    
def c2():
    c()
    c()

def rc():
    g.click(button='right')

def select_all_and_copy():
    rc()
    g.keyDown('a')
    g.keyUp('a')
    g.keyDown('ctrl')
    g.keyDown('c')
    g.keyUp('c')
    g.keyUp('ctrl')  
    
# STEP 1: Open site and Set 4 colors manually
# import sys; import webbrowser; from pandas import read_csv; from time import sleep; webbrowser.get('open -a /Applications/Google\ Chrome.app %s').open("https://floattheturn.com/wp/tools/range-analyzer/")

def change_color(category=1):
    y = 606 + 22*category
    g.moveTo(1156, y)
    c()

def click_hand(hand1, hand2, type_):
    if type_ == "suited" or type_ == "paired":
        x = 765 + 29.9*(14-hand2)
        y = 485 + 29.9*(14-hand1)
    else:
        x = 765 + 29.9*(14-hand1)
        y = 485 + 29.9*(14-hand2)        
    g.moveTo(x, y)
    c()
    
def clear_board():
    change_color(7)
    for i in range(0, 13):
        for j in range(0, 13):
            x = 765 + 29.9*i
            y = 486 + 29.9*j
            g.moveTo(x, y)
            c()
    
def fill_in_dead_cards(flop):
    # Only changing suit for paired cards, don't care that much
    string_chars = []
    suit_to_use_for_paired = ['s','h','c','d']
    index = 0
    for card in flop:
        string_chars.append(m2[card])
        if is_paired and paired_value == card:
            string_chars.append(suit_to_use_for_paired[index])
            string_chars.append(",")
            index+=1
        else:
            string_chars.append("s,")
            
    answer = "".join(string_chars)[:-2] + "c"
    clip.copy(answer)
    g.moveTo(1178, 807)
    c()
    g.hotkey('command', 'a')
    g.hotkey('command', 'v')
    
    
# Change color names
def change_color_names():
    g.moveTo(1246, 628)
    c()
    sleep(0.1)    
    g.hotkey('command', 'a')
    clip.copy("Category 1")
    g.hotkey('command', 'v')
    g.moveTo(1246, 650)
    c()
    sleep(0.1)    
    g.hotkey('command', 'a')
    clip.copy("Category 2")
    g.hotkey('command', 'v')
    g.moveTo(1246, 672)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 3")
    g.hotkey('command', 'v')
    g.moveTo(1246, 694)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 3 FD")
    g.hotkey('command', 'v')
    g.moveTo(1246, 716)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 3 BDFD")
    g.hotkey('command', 'v')   
    g.moveTo(1246, 738)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 4")
    g.hotkey('command', 'v')       

def click_screen():
    g.moveTo(1120, 716)
    c()
    sleep(0.1)

# Create 4 colors
# Name the 4 colors
# click on all the hands for your entire range
# click on cat1 hands
# click on cat2 hands
# click on cat3 hands


click_screen()
change_color_names()

click_screen()
fill_in_dead_cards(flop)

# Fill hands
click_screen()
sleep(0.1)
clear_board()
sleep(0.1)
change_color(6)
sleep(0.1)
[click_hand(x[0], x[1], "paired") for x in my_hands[0]]
[click_hand(x[0], x[1], "suited") for x in my_hands[1]]
[click_hand(x[0], x[1], "offsuit") for x in my_hands[2]]
sleep(0.1)
change_color(1)
[click_hand(x[0][0], x[0][1], "paired") for x in my_hands_cat1[0]]
[click_hand(x[0][0], x[0][1], "suited") for x in my_hands_cat1[1]]
[click_hand(x[0][0], x[0][1], "offsuit") for x in my_hands_cat1[2]]
change_color(2)
sleep(0.1)
[click_hand(x[0][0], x[0][1], "paired") for x in my_hands_cat2[0]]
[click_hand(x[0][0], x[0][1], "suited") for x in my_hands_cat2[1]]
[click_hand(x[0][0], x[0][1], "offsuit") for x in my_hands_cat2[2]]
change_color(3)
sleep(0.1)
# Category 3 regular
[click_hand(x[0][0], x[0][1], "paired") for x in my_hands_cat3[0]]
[click_hand(x[0][0], x[0][1], "suited") for x in my_hands_cat3[1]]
[click_hand(x[0][0], x[0][1], "offsuit") for x in my_hands_cat3[2]]
change_color(4)
sleep(0.1)
# Category 3 Flushdraw
[click_hand(x[0][0], x[0][1], "paired") for x in my_hands_cat3[0] if ((False) if board_type == "rainbow" else (False) if board_type == "two-tone" else (x[1] == 3))]
[click_hand(x[0][0], x[0][1], "suited") for x in my_hands_cat3[1] if ((False) if board_type == "rainbow" else (x[1] in [1,2]) if board_type == "two-tone" else (False))]
[click_hand(x[0][0], x[0][1], "offsuit") for x in my_hands_cat3[2] if ((False) if board_type == "rainbow" else (False) if board_type == "two-tone" else (x[1] == 6))]
change_color(5)
sleep(0.1)
# Category 3 BDFD
[click_hand(x[0][0], x[0][1], "paired") for x in my_hands_cat3[0] if ((False) if board_type == "rainbow" else (x[1] == 3) if board_type == "two-tone" else (False))]
[click_hand(x[0][0], x[0][1], "suited") for x in my_hands_cat3[1] if ((x[1] == 3) if board_type == "rainbow" else (x[1] == 1) if board_type == "two-tone" else (False))]
[click_hand(x[0][0], x[0][1], "offsuit") for x in my_hands_cat3[2] if ((False) if board_type == "rainbow" else (x[1] == 6) if board_type == "two-tone" else (False))]
change_color(7)



Villain SB RFI and You BB 3Bet
Flop: AJT
rainbow
Flop Action: You bet
Turn: 6 offsuit
Turn Action: You raise
River: A offsuit
River Action: Both check
55 33 4 10


In [81]:
my_hands_cat3

[[([8, 8], 3),
  ([7, 7], 3),
  ([6, 6], 3),
  ([5, 5], 3),
  ([4, 4], 3),
  ([2, 2], 3)],
 [([14, 10], 2),
  ([14, 8], 2),
  ([14, 7], 2),
  ([14, 6], 2),
  ([14, 5], 2),
  ([14, 4], 2),
  ([14, 2], 2),
  ([12, 11], 4),
  ([12, 10], 4),
  ([12, 8], 2),
  ([11, 10], 4),
  ([11, 8], 2),
  ([10, 8], 4),
  ([8, 7], 4),
  ([8, 6], 2),
  ([7, 6], 2),
  ([7, 5], 2),
  ([6, 5], 2),
  ([6, 4], 2),
  ([5, 4], 4)],
 [([14, 10], 6),
  ([14, 8], 6),
  ([14, 7], 6),
  ([14, 6], 6),
  ([14, 5], 6),
  ([12, 11], 12),
  ([12, 10], 12),
  ([11, 10], 12)]]

Opponent Range Guessing

In [23]:
opponents_range = "88-22,AJo-A8o,KJo-K8o,QJo-Q8o,JTo-J8o,T9o-T8o,98o,A8s-A2s,K8s-K4s,Q8s-Q4s,J8s-J4s,T8s-T4s,98s-94s,87s-84s,76s-74s,65s-64s,54s-53s,43s-42s,32s"
opponents_hands = range_to_hands(opponents_range)

opponents_position_ip = not my_position_ip
opponents_pfr = not my_pfr

# My hands with combos
opponents_with_combos = [[], [], []]
opponents_with_combos[0] = [(x, 6) if x[0] not in flop else (x, 3) for x in opponents_hands[0]]
opponents_with_combos[1] = [(x, 4) if x[0] not in flop and x[1] not in flop else (x, 2) if x[0] in flop and x[1] in flop else (x,3) for x in opponents_hands[1]]
opponents_with_combos[2] = [(x, 12) if x[0] not in flop and x[1] not in flop else (x, 7) if x[0] in flop and x[1] in flop else (x,9) for x in opponents_hands[2]]


# Important note: lower ranked rules may include higher ranked hands
    # Also tp_j_kicker includes trips because it's okay that it does because of the theory:
    # actions taken by one hand are taken by all better hands within the cat
    # ! Just be careful that you remove cat1 hands from final cat2, same with cat3 with both cat1 & 2
    # Might want to QA with a hand matrix coloring compare with the existing matrix based on default rules

# Cat1
#### Assuming no flushes (monotone boards) for simplicity
opponents_hands_s_straight = [] if is_paired else [(x, 4) for x in opponents_hands[1] if max(x + flop) - min(x + flop) == 4 or max([1 if y == 14 else y for y in (x + flop)]) - min([1 if y == 14 else y for y in (x + flop)]) == 4]
opponents_hands_o_straight = [] if is_paired else [(x, 12) for x in opponents_hands[2] if max(x + flop) - min(x + flop) == 4 or max([1 if y == 14 else y for y in (x + flop)]) - min([1 if y == 14 else y for y in (x + flop)]) == 4]
opponents_hands_pp_sets = [(x, 3) for x in opponents_hands[0] if x[0] in flop]
opponents_hands_s_trips = [] if not is_paired else [(x, 2) for x in opponents_hands[1] if x[0] == paired_value or x[1] == paired_value]
opponents_hands_o_trips = [] if not is_paired else [(x, 6) for x in opponents_hands[2] if x[0] == paired_value or x[1] == paired_value]
# 2 combos most times, not 3; 7 more often than 6
opponents_hands_s_two_pair = [] if is_paired else [(x, 2) for x in opponents_hands[1] if x[0] in flop and x[1] in flop]
opponents_hands_o_two_pair = [] if is_paired else [(x, 7) for x in opponents_hands[2] if x[0] in flop and x[1] in flop]
opponents_hands_pp_overpair_9plus = [(x, 6) for x in opponents_hands[0] if x[0] > flop[0] and x[0] >= 9]
opponents_hands_pp_any_overpair = [(x, 6) for x in opponents_hands[0] if x[0] > flop[0]]
opponents_hands_s_tp_k_kicker = [(x, 3) for x in opponents_hands[1] if (x[0] == flop[0] and x[1] >= 13) or (x[1] == flop[0] and x[0] >= 13)]
opponents_hands_o_tp_k_kicker = [(x, 9) for x in opponents_hands[2] if (x[0] == flop[0] and x[1] >= 13) or (x[1] == flop[0] and x[0] >= 13)]
opponents_hands_s_tp_j_kicker = [(x, 3) for x in opponents_hands[1] if (x[0] == flop[0] and x[1] >= 11 and x[1] <= 12 and x[1] not in flop) or (x[1] == flop[0] and x[0] >= 11 and x[0] <= 12 and x[0] not in flop)]
opponents_hands_o_tp_j_kicker = [(x, 9) for x in opponents_hands[2] if (x[0] == flop[0] and x[1] >= 11 and x[1] <= 12 and x[1] not in flop) or (x[1] == flop[0] and x[0] >= 11 and x[0] <= 12 and x[0] not in flop)]
opponents_hands_s_tp_any_kicker = [(x, 3) for x in opponents_hands[1] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]
opponents_hands_o_tp_any_kicker = [(x, 9) for x in opponents_hands[2] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]

# Cat2 (flushdraws with high card hand might actually be part of cat3, but saying the combos are part of cat2)
opponents_hands_s_tp_bad_kicker = [(x, 3) for x in opponents_hands[1] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]
opponents_hands_o_tp_bad_kicker = [(x, 9) for x in opponents_hands[2] if (x[0] == flop[0] and x[1] <= 10 and x[1] not in flop) or (x[1] == flop[0] and x[0] <= 10 and x[0] not in flop)]
opponents_hands_s_middle_pair = [(x, 3) for x in opponents_hands[1] if (x[0] == flop[1] and x[1] not in flop) or (x[1] == flop[1] and x[0] not in flop)]
opponents_hands_o_middle_pair = [(x, 9) for x in opponents_hands[2] if (x[0] == flop[1] and x[1] not in flop) or (x[1] == flop[1] and x[0] not in flop)]
opponents_hands_s_bottom_pair = [(x, 3) for x in opponents_hands[1] if (x[0] == flop[2] and x[1] not in flop) or (x[1] == flop[2] and x[0] not in flop)]
opponents_hands_o_bottom_pair = [(x, 9) for x in opponents_hands[2] if (x[0] == flop[2] and x[1] not in flop) or (x[1] == flop[2] and x[0] not in flop)]
opponents_hands_pp_below_top_pair = [(x, 6) for x in opponents_hands[0] if x[0] < flop[0] and x[0] > flop[1]]
opponents_hands_s_aj_high = [(x, 4) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and (x[0] == 14) and (x[1] > 10)]
opponents_hands_o_aj_high = [(x, 12) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and (x[0] == 14) and (x[1] > 10)]
opponents_hands_pp_below_middle_pair = [(x, 6) for x in opponents_hands[0] if x[0] < flop[1] and x[0] > flop[2]]
opponents_hands_s_kq_high = [(x, 4) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and ((x[0] == 13 and x[1] > 11) or (x[0] == 14))]
opponents_hands_o_kq_high = [(x, 12) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and ((x[0] == 13 and x[1] > 11) or (x[0] == 14))]
opponents_hands_pp_below_bottom_pair = [(x, 6) for x in opponents_hands[0] if x[0] < flop[2]]
opponents_hands_s_kj_high = [(x, 4) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] == 11)]
opponents_hands_o_kj_high = [(x, 12) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] == 11)]
opponents_hands_s_k8_high = [(x, 4) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] < 11 and x[1] >= 8)]
opponents_hands_o_k8_high = [(x, 12) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and (x[0] == 13) and (x[1] < 11 and x[1] >= 8)]

# Cat3 paired
#### bdfd/fd/no_fd combos should not be counted twice; combine them smartly and carefully.
#### Assuming no suited flushdraw possible with pair on two-tone for simplicity
#### Might say a straight is a gutshot but that is fine because future logic
## Include the combos for that hand
if board_type == "two-tone":
    opponents_hands_pp_fd = []
    opponents_hands_s_fd = [(x, 1) for x in opponents_hands[1] if x[0] not in flop and x[1] not in flop]
    opponents_hands_o_fd = []
elif board_type == "rainbow":
    opponents_hands_pp_fd = []
    opponents_hands_s_fd = []
    opponents_hands_o_fd = []
else:
    opponents_hands_pp_fd = [(x, 3) for x in opponents_hands[0] if x[0] not in flop]
    opponents_hands_s_fd = []
    # If paired then ignore the flushdraw (anyway, just monotone; just makes things simpler)
    opponents_hands_o_fd = [(x, 6) for x in opponents_hands[2] if x[0] not in flop and x[1] not in flop]
#### Also added double gutshots.  Doing a bit of overcounting for oesd+pair, which is fine for estimation (should be 9).
opponents_hands_pp_oesd = [(x, 6) for x in opponents_hands[0] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 3) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 3) or (sorted(x+flop) == [3,4,5,7,14]) or (max(x+flop) - min(x+flop) == 6 and min(x+flop)+2 == sorted(x+flop)[1] and max(x+flop)-2 == sorted(x+flop)[-2])]
opponents_hands_s_oesd = [(x, 4) for x in opponents_hands[1] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 3) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 3) or (sorted(x+flop) == [3,4,5,7,14]) or (max(x+flop) - min(x+flop) == 6 and min(x+flop)+2 == sorted(x+flop)[1] and max(x+flop)-2 == sorted(x+flop)[-2])]
opponents_hands_o_oesd = [(x, 12) for x in opponents_hands[2] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 3) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 3) or (sorted(x+flop) == [3,4,5,7,14]) or (max(x+flop) - min(x+flop) == 6 and min(x+flop)+2 == sorted(x+flop)[1] and max(x+flop)-2 == sorted(x+flop)[-2])]
opponents_hands_pp_gutshot = [(x, 6) for x in opponents_hands[0] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 4) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 4) or (sorted(set([1 if y == 14 else y for y in (x + flop)] + [20,21,22]))[3] <= 5)]
opponents_hands_s_gutshot = [(x, 4) for x in opponents_hands[1] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 4) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 4) or (sorted(set([1 if y == 14 else y for y in (x + flop)] + [20,21,22]))[3] <= 5)]
opponents_hands_o_gutshot = [(x, 12) for x in opponents_hands[2] if (sorted(set(x + flop + [20,21,22]))[3] - min(x + flop) == 4) or (max(x + flop) - sorted(set(x + flop + [-20,-19,-18]))[-4] == 4) or (sorted(set([1 if y == 14 else y for y in (x + flop)] + [20,21,22]))[3] <= 5)]
#### Additional rule: 3 to a straight requires two cards from your hand not just one (that's how I want it to be)
opponents_hands_s_3_to_straight_not_all_from_low_end = [(x, 4) for x in opponents_hands[1] if (x[0] != 14) and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]+1 and x[1] == flop[0]-1) or (x[0] == flop[0]+2 and x[1] == flop[0]+1) or (x[0] == flop[1]+1 and x[1] == flop[1]-1) or (x[0] == flop[1]+2 and x[1] == flop[1]+1) or (x[0] == flop[2]+1 and x[1] == flop[2]-1) or (x[0] == flop[2]+2 and x[1] == flop[2]+1))]
opponents_hands_o_3_to_straight_not_all_from_low_end = [(x, 12) for x in opponents_hands[2] if (x[0] != 14) and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]+1 and x[1] == flop[0]-1) or (x[0] == flop[0]+2 and x[1] == flop[0]+1) or (x[0] == flop[1]+1 and x[1] == flop[1]-1) or (x[0] == flop[1]+2 and x[1] == flop[1]+1) or (x[0] == flop[2]+1 and x[1] == flop[2]-1) or (x[0] == flop[2]+2 and x[1] == flop[2]+1))]
if board_type == "two-tone":
    opponents_hands_s_3_to_straight_low_end_bdfd = [(x, 1) for x in opponents_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    opponents_hands_o_3_to_straight_low_end_bdfd = [(x, 6) for x in opponents_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
elif board_type == "rainbow":
    opponents_hands_s_3_to_straight_low_end_bdfd = [(x, 3) for x in opponents_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    opponents_hands_o_3_to_straight_low_end_bdfd = []
else:
    opponents_hands_s_3_to_straight_low_end_bdfd = []
    opponents_hands_o_3_to_straight_low_end_bdfd = []
if board_type == "two-tone":
    opponents_hands_s_3_to_straight_low_end = [(x, 4) for x in opponents_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    opponents_hands_o_3_to_straight_low_end = [(x, 12) for x in opponents_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
elif board_type == "rainbow":
    opponents_hands_s_3_to_straight_low_end = [(x, 4) for x in opponents_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    opponents_hands_o_3_to_straight_low_end = [(x, 12) for x in opponents_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
else:
    opponents_hands_s_3_to_straight_low_end = [(x, 4) for x in opponents_hands[1] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
    opponents_hands_o_3_to_straight_low_end = [(x, 12) for x in opponents_hands[2] if x[0] != 13 and (x[0] not in flop and x[1] not in flop) and ((x[0] == flop[0]-1 and x[1] == flop[0]-2) or (x[0] == flop[1]-1 and x[1] == flop[1]-2) or (x[0] == flop[2]-1 and x[1] == flop[2]-2))]
if board_type == "two-tone":
    opponents_hands_s_5_unique_cards_within_7_values_bdfd = [(x, 1) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and max(flop+x) - min(flop+x) <= 7]
    opponents_hands_o_5_unique_cards_within_7_values_bdfd = [(x, 6) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and max(flop+x) - min(flop+x) <= 7]
elif board_type == "rainbow":
    opponents_hands_s_5_unique_cards_within_7_values_bdfd = [(x, 3) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and max(flop+x) - min(flop+x) <= 7]
    opponents_hands_o_5_unique_cards_within_7_values_bdfd = []
else:
    opponents_hands_s_5_unique_cards_within_7_values_bdfd = []
    opponents_hands_o_5_unique_cards_within_7_values_bdfd = []
if board_type == "two-tone":
    opponents_hands_pp_q_minus_bdfd = [(x, 3) for x in opponents_hands[0] if (x[0] not in flop) and x[0] <= 12]
    opponents_hands_s_q_minus_bdfd = [(x, 1) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] <= 12]
    opponents_hands_o_q_minus_bdfd = [(x, 6) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and x[0] <= 12]
elif board_type == "rainbow":
    opponents_hands_pp_q_minus_bdfd = []
    opponents_hands_s_q_minus_bdfd = [(x, 3) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] <= 12]
    opponents_hands_o_q_minus_bdfd = []
else:
    opponents_hands_pp_q_minus_bdfd = []
    opponents_hands_s_q_minus_bdfd = []
    opponents_hands_o_q_minus_bdfd = []
#### 3 cards within 4 values with two overcards
opponents_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards = [(x, 4) for x in opponents_hands[1] if x[1] > flop[0] and ((max(flop + [x[1]]) - sorted(set(flop + [x[1]] + [-20,-19,-18]))[-3] <= 3) or (max(flop + x) - sorted(set(flop + x + [-20,-19,-18]))[-3] <= 3))]
opponents_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards = [(x, 12) for x in opponents_hands[2] if x[1] > flop[0] and ((max(flop + [x[1]]) - sorted(set(flop + [x[1]] + [-20,-19,-18]))[-3] <= 3) or (max(flop + x) - sorted(set(flop + x + [-20,-19,-18]))[-3] <= 3))]
if board_type == "two-tone":
    opponents_hands_pp_a_minus_bdfd = [(x, 3) for x in opponents_hands[0] if (x[0] not in flop) and x[0] > 12 and x[0] <= 14]
    opponents_hands_s_a_minus_bdfd = [(x, 1) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] > 12 and x[0] <= 14]
    opponents_hands_o_a_minus_bdfd = [(x, 6) for x in opponents_hands[2] if (x[0] not in flop and x[1] not in flop) and x[0] > 12 and x[0] <= 14]
elif board_type == "rainbow":
    opponents_hands_pp_a_minus_bdfd = []
    opponents_hands_s_a_minus_bdfd = [(x, 3) for x in opponents_hands[1] if (x[0] not in flop and x[1] not in flop) and x[0] > 12 and x[0] <= 14]
    opponents_hands_o_a_minus_bdfd = []
else:
    opponents_hands_pp_a_minus_bdfd = []
    opponents_hands_s_a_minus_bdfd = []
    opponents_hands_o_a_minus_bdfd = []



def opponents_cat1_level_x_and_above(x):
    result = [[], [], []]
    if x >= 1:
        result[1] += opponents_hands_s_straight
        result[2] += opponents_hands_o_straight
    if x >= 2:
        result[0] += opponents_hands_pp_sets
    if x >= 3:
        result[1] += opponents_hands_s_trips
        result[2] += opponents_hands_o_trips
    if x >= 4:
        result[1] += opponents_hands_s_two_pair
        result[2] += opponents_hands_o_two_pair
    if x >= 5:
        result[0] += opponents_hands_pp_overpair_9plus
    if x >= 6:
        result[0] += opponents_hands_pp_any_overpair
    if x >= 7:
        result[1] += opponents_hands_s_tp_k_kicker
        result[2] += opponents_hands_o_tp_k_kicker
    if x >= 8:
        result[1] += opponents_hands_s_tp_j_kicker
        result[2] += opponents_hands_o_tp_j_kicker
    if x >= 9:
        result[1] += opponents_hands_s_tp_any_kicker
        result[2] += opponents_hands_o_tp_any_kicker

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Return result
    opponents_hands_cat1 = result
    return opponents_hands_cat1

# Performance improvement by filtering out cat1 from hands already, but would also need a copy of hands
def opponents_cat2_level_x_and_above(x, opponents_hands_cat1):
    result = [[], [], []]
    if x >= 1:
        # Cat 1
        result[1] += opponents_hands_s_straight
        result[2] += opponents_hands_o_straight
        result[0] += opponents_hands_pp_sets
        result[1] += opponents_hands_s_trips
        result[2] += opponents_hands_o_trips
        result[1] += opponents_hands_s_two_pair
        result[2] += opponents_hands_o_two_pair
        result[0] += opponents_hands_pp_overpair_9plus
        result[0] += opponents_hands_pp_any_overpair
        result[1] += opponents_hands_s_tp_k_kicker
        result[2] += opponents_hands_o_tp_k_kicker
        result[1] += opponents_hands_s_tp_j_kicker
        result[2] += opponents_hands_o_tp_j_kicker
        result[1] += opponents_hands_s_tp_any_kicker
        result[2] += opponents_hands_o_tp_any_kicker

        # Cat 2
        result[1] += opponents_hands_s_tp_bad_kicker
        result[2] += opponents_hands_o_tp_bad_kicker
    if x >= 2:
        result[1] += opponents_hands_s_middle_pair
        result[2] += opponents_hands_o_middle_pair
    if x >= 3:
        result[0] += opponents_hands_pp_below_top_pair
    if x >= 4:
        result[1] += opponents_hands_s_bottom_pair
        result[2] += opponents_hands_o_bottom_pair
    if x >= 5:
        result[1] += opponents_hands_s_aj_high
        result[2] += opponents_hands_o_aj_high
    if x >= 6:
        result[0] += opponents_hands_pp_below_middle_pair
    if x >= 7:
        result[1] += opponents_hands_s_kq_high
        result[2] += opponents_hands_o_kq_high
    if x >= 8:
        result[0] += opponents_hands_pp_below_bottom_pair
    if x >= 9:
        result[1] += opponents_hands_s_kj_high
        result[2] += opponents_hands_o_kj_high
    if x >= 10:
        result[1] += opponents_hands_s_k8_high
        result[2] += opponents_hands_o_k8_high

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Interim
    cat1_unique_pp = [x for (x,y) in opponents_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in opponents_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in opponents_hands_cat1[2]]

    # Remove cat1 from these cat2s
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o]

    # Return result
    opponents_hands_cat2 = result
    return opponents_hands_cat2

# Performance improvement by filtering out cat1+cat2 from hands already, but would also need a copy of hands
def opponents_cat3_level_x_and_above(x, opponents_hands_cat1, opponents_hands_cat2, skip_4_to_10_and_13_to_15=True):
    bdfd_result = [[], [], []]
    other_result = [[], [], []]
    result = [[], [], []]
    if x >= 1:
        other_result[0] += opponents_hands_pp_fd
        other_result[1] += opponents_hands_s_fd
        other_result[2] += opponents_hands_o_fd
    if x >= 2:
        other_result[0] += opponents_hands_pp_oesd
        other_result[1] += opponents_hands_s_oesd
        other_result[2] += opponents_hands_o_oesd
    if x >= 3:
        other_result[0] += opponents_hands_pp_gutshot
        other_result[1] += opponents_hands_s_gutshot
        other_result[2] += opponents_hands_o_gutshot
    if x >= 4 and not skip_4_to_10_and_13_to_15:
        other_result[1] += opponents_hands_s_3_to_straight_not_all_from_low_end
        other_result[2] += opponents_hands_o_3_to_straight_not_all_from_low_end
    if x >= 5 and not skip_4_to_10_and_13_to_15:
        bdfd_result[1] += opponents_hands_s_3_to_straight_low_end_bdfd
        bdfd_result[2] += opponents_hands_o_3_to_straight_low_end_bdfd
    if x >= 6 and not skip_4_to_10_and_13_to_15:
        other_result[1] += opponents_hands_s_3_to_straight_low_end
        other_result[2] += opponents_hands_o_3_to_straight_low_end
    if x >= 7 and not skip_4_to_10_and_13_to_15:
        bdfd_result[1] += opponents_hands_s_5_unique_cards_within_7_values_bdfd
        bdfd_result[2] += opponents_hands_o_5_unique_cards_within_7_values_bdfd
    if x >= 8 and not skip_4_to_10_and_13_to_15:
        bdfd_result[0] += opponents_hands_pp_q_minus_bdfd
        bdfd_result[1] += opponents_hands_s_q_minus_bdfd
        bdfd_result[2] += opponents_hands_o_q_minus_bdfd
    if x >= 9:
        other_result[1] += opponents_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
        other_result[2] += opponents_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
    if x >= 10 and not skip_4_to_10_and_13_to_15:
        bdfd_result[0] += opponents_hands_pp_a_minus_bdfd
        bdfd_result[1] += opponents_hands_s_a_minus_bdfd
        bdfd_result[2] += opponents_hands_o_a_minus_bdfd

    # Remove duplicates within bdfd hands
    bdfd_result[0].sort(reverse=True)
    bdfd_result[1].sort(reverse=True)
    bdfd_result[2].sort(reverse=True)
    bdfd_result[0] = list(k for k,_ in itertools.groupby(bdfd_result[0]))
    bdfd_result[1] = list(k for k,_ in itertools.groupby(bdfd_result[1]))
    bdfd_result[2] = list(k for k,_ in itertools.groupby(bdfd_result[2]))

    # Add all together
    result[0] = bdfd_result[0] + other_result[0]
    result[1] = bdfd_result[1] + other_result[1]
    result[2] = bdfd_result[2] + other_result[2]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] += val[1]
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] += val[1]
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] += val[1]
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Interim
    cat1_unique_pp = [x for (x,y) in opponents_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in opponents_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in opponents_hands_cat1[2]]
    cat2_unique_pp = [x for (x,y) in opponents_hands_cat2[0]]
    cat2_unique_s = [x for (x,y) in opponents_hands_cat2[1]]
    cat2_unique_o = [x for (x,y) in opponents_hands_cat2[2]]

    # Remove cat1 and cat2
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp and x not in cat2_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s and x not in cat2_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o and x not in cat2_unique_o]

    # Add cat2 hands
    if x >= 11 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_k8_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_k8_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 12 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_kj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_kj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 13 and not skip_4_to_10_and_13_to_15:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_bottom_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 14 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_kq_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_kq_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 15 and not skip_4_to_10_and_13_to_15:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_middle_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    # Add cat4 hands
    if x >= 16:
        remaining_cat2_type_hands_pp = [x for (x,y) in opponents_hands_pp_below_bottom_pair] + [x for (x,y) in opponents_hands_pp_below_middle_pair] + [x for (x,y) in opponents_hands_pp_below_top_pair]
        remaining_cat2_type_hands_s = [x for (x,y) in opponents_hands_s_k8_high] + [x for (x,y) in opponents_hands_s_kj_high] + [x for (x,y) in opponents_hands_s_kq_high] + [x for (x,y) in opponents_hands_s_aj_high] + [x for (x,y) in opponents_hands_s_bottom_pair] + [x for (x,y) in opponents_hands_s_middle_pair] + [x for (x,y) in opponents_hands_s_tp_bad_kicker]
        remaining_cat2_type_hands_o = [x for (x,y) in opponents_hands_o_k8_high] + [x for (x,y) in opponents_hands_o_kj_high] + [x for (x,y) in opponents_hands_o_kq_high] + [x for (x,y) in opponents_hands_o_aj_high] + [x for (x,y) in opponents_hands_o_bottom_pair] + [x for (x,y) in opponents_hands_o_middle_pair] + [x for (x,y) in opponents_hands_o_tp_bad_kicker]
        result[0] += [(x, 6) for x in opponents_hands[0] if x not in cat1_unique_pp and x not in cat2_unique_pp and x not in remaining_cat2_type_hands_pp]
        result[1] += [(x, 4) for x in opponents_hands[1] if x not in cat1_unique_s and x not in cat2_unique_s and x not in remaining_cat2_type_hands_s]
        result[2] += [(x, 12) for x in opponents_hands[2] if x not in cat1_unique_o and x not in cat2_unique_o and x not in remaining_cat2_type_hands_o]
    # Add cat2 hands with pairs
    if x >= 17:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_aj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_aj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 18:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_bottom_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_bottom_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 19:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_top_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 20:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_middle_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_middle_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 21:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_tp_bad_kicker if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_tp_bad_kicker if x not in cat1_unique_o and x not in cat2_unique_o]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Return results
    opponents_hands_cat3 = result
    return opponents_hands_cat3


    
    
    
    
    
    
    
best_param2 = 16 if opponents_position_ip and opponents_pfr else 2
    
    
opponents_cat1 = opponents_cat1_level_x_and_above(8)
# RANGE FROM 5-10:
opponents_cat2 = opponents_cat2_level_x_and_above(7, opponents_cat1)
# CHOICES 1, 2, 5, 8, 10, 16:
opponents_cat3 = opponents_cat3_level_x_and_above(best_param2, opponents_cat1, opponents_cat2)      
opponents_cat4 = get_cat4_hands(opponents_with_combos, opponents_cat1, opponents_cat2, opponents_cat3)
cat1_combos = count_hand_combos(opponents_cat1)
cat2_combos = count_hand_combos(opponents_cat2)
cat3_combos = count_hand_combos(opponents_cat3)
cat4_combos = count_hand_combos(opponents_cat4)
print(cat1_combos, cat2_combos, cat3_combos, cat4_combos)
# print("cat3 combos needed:", cat1_combos*2 - cat3_combos)
# print("cat2 combos needed:", cat4_combos - cat2_combos)
print("Weakest bluff to use:", best_param2)



import re
import pyperclip as clip
import pyautogui as g
from time import sleep
import pyperclip as clip
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
g.FAILSAFE = True
sleep(2)
g.PAUSE = 0.0
g.size()
g.position()

def c():
    g.mouseDown()
    g.mouseUp()
    
def c2():
    c()
    c()

def rc():
    g.click(button='right')

def select_all_and_copy():
    rc()
    g.keyDown('a')
    g.keyUp('a')
    g.keyDown('ctrl')
    g.keyDown('c')
    g.keyUp('c')
    g.keyUp('ctrl')  
    
# STEP 1: Open site and Set 4 colors manually
# import sys; import webbrowser; from pandas import read_csv; from time import sleep; webbrowser.get('open -a /Applications/Google\ Chrome.app %s').open("https://floattheturn.com/wp/tools/range-analyzer/")

def change_color(category=1):
    y = 606 + 22*category
    g.moveTo(1156, y)
    c()

def click_hand(hand1, hand2, type_):
    if type_ == "suited" or type_ == "paired":
        x = 765 + 29.9*(14-hand2)
        y = 485 + 29.9*(14-hand1)
    else:
        x = 765 + 29.9*(14-hand1)
        y = 485 + 29.9*(14-hand2)        
    g.moveTo(x, y)
    c()
    
def clear_board():
    change_color(7)
    for i in range(0, 13):
        for j in range(0, 13):
            x = 765 + 29.9*i
            y = 486 + 29.9*j
            g.moveTo(x, y)
            c()
    
def fill_in_dead_cards(flop):
    # Only changing suit for paired cards, don't care that much
    string_chars = []
    suit_to_use_for_paired = ['s','h','c','d']
    index = 0
    for card in flop:
        string_chars.append(m2[card])
        if is_paired and paired_value == card:
            string_chars.append(suit_to_use_for_paired[index])
            string_chars.append(",")
            index+=1
        else:
            string_chars.append("s,")
            
    answer = "".join(string_chars)[:-2] + "c"
    clip.copy(answer)
    g.moveTo(1178, 807)
    c()
    g.hotkey('command', 'a')
    g.hotkey('command', 'v')
    
    
# Change color names
def change_color_names():
    g.moveTo(1246, 628)
    c()
    sleep(0.1)    
    g.hotkey('command', 'a')
    clip.copy("Category 1")
    g.hotkey('command', 'v')
    g.moveTo(1246, 650)
    c()
    sleep(0.1)    
    g.hotkey('command', 'a')
    clip.copy("Category 2")
    g.hotkey('command', 'v')
    g.moveTo(1246, 672)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 3")
    g.hotkey('command', 'v')
    g.moveTo(1246, 694)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 3 FD")
    g.hotkey('command', 'v')
    g.moveTo(1246, 716)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 3 BDFD")
    g.hotkey('command', 'v')   
    g.moveTo(1246, 738)
    c()
    sleep(0.1)
    g.hotkey('command', 'a')
    clip.copy("Category 4")
    g.hotkey('command', 'v')       

def click_screen():
    g.moveTo(1120, 716)
    c()
    sleep(0.1)

# Create 4 colors
# Name the 4 colors
# click on all the hands for your entire range
# click on cat1 hands
# click on cat2 hands
# click on cat3 hands


click_screen()
change_color_names()

click_screen()
fill_in_dead_cards(flop)

# Fill hands
click_screen()
sleep(0.1)
clear_board()
sleep(0.1)
change_color(6)
sleep(0.1)
[click_hand(x[0], x[1], "paired") for x in opponents[0]]
[click_hand(x[0], x[1], "suited") for x in opponents[1]]
[click_hand(x[0], x[1], "offsuit") for x in opponents[2]]
sleep(0.1)
change_color(1)
[click_hand(x[0][0], x[0][1], "paired") for x in opponents_cat1[0]]
[click_hand(x[0][0], x[0][1], "suited") for x in opponents_cat1[1]]
[click_hand(x[0][0], x[0][1], "offsuit") for x in opponents_cat1[2]]
change_color(2)
sleep(0.1)
[click_hand(x[0][0], x[0][1], "paired") for x in opponents_cat2[0]]
[click_hand(x[0][0], x[0][1], "suited") for x in opponents_cat2[1]]
[click_hand(x[0][0], x[0][1], "offsuit") for x in opponents_cat2[2]]
change_color(3)
sleep(0.1)
# Category 3 regular
[click_hand(x[0][0], x[0][1], "paired") for x in opponents_cat3[0]]
[click_hand(x[0][0], x[0][1], "suited") for x in opponents_cat3[1]]
[click_hand(x[0][0], x[0][1], "offsuit") for x in opponents_cat3[2]]
change_color(4)
sleep(0.1)
# Category 3 Flushdraw
[click_hand(x[0][0], x[0][1], "paired") for x in opponents_cat3[0] if ((False) if board_type == "rainbow" else (False) if board_type == "two-tone" else (x[1] == 3))]
[click_hand(x[0][0], x[0][1], "suited") for x in opponents_cat3[1] if ((False) if board_type == "rainbow" else (x[1] in [1,2]) if board_type == "two-tone" else (False))]
[click_hand(x[0][0], x[0][1], "offsuit") for x in opponents_cat3[2] if ((False) if board_type == "rainbow" else (False) if board_type == "two-tone" else (x[1] == 6))]
change_color(5)
sleep(0.1)
# Category 3 BDFD
[click_hand(x[0][0], x[0][1], "paired") for x in opponents_cat3[0] if ((False) if board_type == "rainbow" else (x[1] == 3) if board_type == "two-tone" else (False))]
[click_hand(x[0][0], x[0][1], "suited") for x in opponents_cat3[1] if ((x[1] == 3) if board_type == "rainbow" else (x[1] == 1) if board_type == "two-tone" else (False))]
[click_hand(x[0][0], x[0][1], "offsuit") for x in opponents_cat3[2] if ((False) if board_type == "rainbow" else (x[1] == 6) if board_type == "two-tone" else (False))]
change_color(7)




105 105 44 174
Weakest bluff to use: 2


NameError: name 'opponents' is not defined